In [1]:
import sys, os
sys.path.append('./modules')
from Monitoring_Tools import *
from OpenStack_Tools  import *

In [2]:
def platformStatus(platform):
    HTML_OP=''
    
    show_notebook_url(platform, host_ip="10.3.216.210", port=8888)
    HTML_OP += '<b>' + html_platform_info(platform)+ '</b>'
    
    ini_file="$HOME/env/{}_hosts.ini".format(platform)
    print("Reading " + ini_file)

    inventory = read_inventory(ini_file)
    
    print("Connecting to cloud: " + platform)
    conn = connectToCloud(platform)
    
    print("Pinging ...")
    HTML, PING_STATUS = html_ping_all(inventory)
    HTML_OP += HTML
    
    print("Get server/flavor/image list ...")
    HTML, VMS_STATUS = getServerList( conn )
    HTML_OP += HTML
    
    print("Pinging ports ...")
    HTML, PING_PORTS_STATUS = html_ping_ports_all(inventory)
    HTML_OP += HTML
    
    print("Pinging endpoints ...")
    HTML, ENDPOINTS_STATUS = html_endpoint_urls(conn)
    HTML_OP += HTML
    
    #archive_df(inventory)
    return HTML_OP, PING_STATUS, VMS_STATUS, PING_PORTS_STATUS, ENDPOINTS_STATUS

platforms = ['demo8', 'poc1', 'poc2', 'nfv5']
#platforms = ['demo8', 'poc1' ]

HTML_OPS={}
STATUSES={}
for platform in platforms:
    HTML_OP, PING_STATUS, VMS_STATUS, PING_PORTS_STATUS, ENDPOINTS_STATUS = platformStatus(platform)
    HTML_OPS[platform] = HTML_OP
    STATUSES[platform] = [ PING_STATUS, VMS_STATUS, PING_PORTS_STATUS, ENDPOINTS_STATUS ]

Reading $HOME/env/demo8_hosts.ini
Connecting to cloud: demo8
Pinging ...
Get server/flavor/image list ...
2 flavors
7 images
2 servers
Pinging ports ...
ping(demo8co0[10.3.162.143:22]) ... ping(demo8co1[10.3.162.141:22]) ... ping(demo8co10[10.3.162.146:22]) ... ping(demo8co11[10.3.162.140:22]) ... ping(demo8co12[10.3.162.139:22]) ... ping(demo8co13[10.3.162.134:22]) ... ping(demo8co14[10.3.162.144:22]) ... ping(demo8co15[10.3.162.145:22]) ... ping(demo8co2[10.3.162.178:22]) ... ping(demo8co3[10.3.162.142:22]) ... ping(demo8co4[10.3.162.149:22]) ... ping(demo8co5[10.3.162.137:22]) ... ping(demo8co6[10.3.162.138:22]) ... ping(demo8co7[10.3.162.153:22]) ... ping(demo8co8[10.3.162.154:22]) ... ping(demo8co9[10.3.162.136:22]) ... ping(hlmvm[10.3.162.30:22]) ... ping(kvm[10.3.162.10:22]) ... ping(wrcl0[10.3.162.131:22]) ... ping(wrcl1[10.3.162.132:22]) ... Pinging endpoints ...


Reading $HOME/env/poc1_hosts.ini
Connecting to cloud: poc1
Pinging ...
ping(wrcllr0[10.3.30.21]) ... ping(wrcllr1[10.3.30.22]) ... Get server/flavor/image list ...
8 flavors
25 images
10 servers
Pinging ports ...
ping(hlm1[10.3.28.46:22]) ... ping(hlm1vm[10.3.29.10:22]) ... ping(wrcllr0[10.3.30.21:22]) ... ping(wrcllr1[10.3.30.22:22]) ... Pinging endpoints ...


In [10]:
#print(STATUSES)
display( HTML( '<h1>Platforms Status Summary </h1>' ))
DISP_STATUSES={}
for platform in platforms:
    LINK='<a href="#RESULTS_' + platform + '">' + platform +'</a>'
    #print(platform + " " + LINK)
    DISP_STATUSES[LINK]=STATUSES[platform]
#print(DISP_STATUSES)  


error_highlight='<div style="background-color: red; color: white"><b>{}</b></div>'
warn_highlight='<div style="background-color: orange; color: white"><b>{}</b></div>'
ok_highlight='<div style="background-color: green; color: white"><b>{}</b></div>'

highlights={
    'OK':      ok_highlight,
    'WARN':  warn_highlight,
    'ERROR': error_highlight,
}

display( HTML( DictTable._repr_html_(DISP_STATUSES, highlights) ))

display( HTML( '<h1>Platforms results </h1>' ))
for platform in platforms:
    display( HTML( '<a name="RESULTS_{}"></a>'.format(platform) ))
    #display( HTML( '<h1><a name="{}">Platform {} results</a></h1>'.format(platform, platform) ))
    HTML_OP = HTML_OPS[platform]
    display( HTML( HTML_OP )) 

demo8,OK,OK,OK,OK
poc1,OK,OK,OK,OK


In [4]:
def archive_df(inventory):
    ! [ ! -d history ] && mkdir history

    import datetime
    import time

    #d = datetime.date.today().strftime("%B %d, %Y")
    #dt = datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")
    d = datetime.date.today().strftime("%Y-%m-%d")
    dt = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M")
    print(d)
    print(dt)

    for host in sorted(inventory['df_check']):    
        ip = inventory['hosts'][host]['ansible_host']
        user = inventory['hosts'][host]['ansible_user']
        pkey = inventory['hosts'][host]['ssh_key']
        df_check = inventory['hosts'][host]['df_check']

        # write to history subdir (~/notebooks/cron for cron jobs)
        history_file='history/df_history_' + platform + '_' + host + '.txt'
        history_fd = open(history_file, 'a')

        full_df_cmd="hostname; df 2>&1"
        df_op, stderr = ssh_command(host, ip, user, pkey, full_df_cmd)    
        history_fd.write('DATE:' + dt + '\n' + df_op)
        history_fd.close()

        df_cmd="df " + df_check.replace(",", " ") + "| grep -v ^Filesystem"
        df_op, stderr = ssh_command(host, ip, user, pkey, df_cmd)    
        #df_op = stdout.decode('utf8')
        #print("HOST[" + host + "]<" + df_check + ">{" + df_cmd +"}:" + df_op)

        df_lines=df_op.split("\n")
        for df_line in df_lines:
            #print("LINE: " + df_line)
            pc_pos = df_line.find("%")
            if pc_pos != -1:
                pc=int(df_line[pc_pos-3:pc_pos])
                partn=df_line[pc_pos+1:]
                print(host + " " + str(pc) + "% " + partn)


In [5]:
def showUptimes():
    for host in sorted(inventory['ssh_check']):    
        ip = inventory['hosts'][host]['ansible_host']
        user = inventory['hosts'][host]['ansible_user']
        pkey = inventory['hosts'][host]['ssh_key']

        stdout, stderr = ssh_command(host, ip, user, pkey, "uptime")

        #print("LINE=" + stdout)
        uptime = strip_uptime(stdout)
        print(host + ":" + uptime)

### TODO:
- Create status at top of page (without Javascript - so visible in email?)
- df processing to show trends (as html table, as pie chart)
- use of openstack clientmanager for equivalent to "openstack service list" status

In [6]:
# TODO: 
# equivalent of openstack service list

# See: how service list is implemented:
#    https://github.com/openstack/python-openstackclient/blob/master/openstackclient/compute/v2/service.py
# (compute_client = client_manager.compute)
#
# See also: how to create/use a clientmanager:
#    https://github.com/openstack/python-openstackclient/blob/master/examples/osc-lib.py

#from openstackclient.common import clientmanager
#from openstackclient.common import utils